In [1]:
import json
import pandas as pd

In [32]:
# Load cleaned song meta data(79,814)
with open('./cleaned_song_meta.json', 'r', encoding='UTF8') as f:
    data = json.load(f)
song_df = pd.json_normalize(data)

# id, song_name, artist_name_basket
song_df = song_df[['id', 'song_name', 'artist_name_basket']]
song_df['tags'] = [[] for _ in range(len(song_df))]
song_df['count'] = 0

In [33]:
song_df

,id,song_name,artist_name_basket,tags,count
0,9,"사랑, 그대라는 멜로디",[진호],[],0
1,17,Girl Crush,[마마무 (Mamamoo)],[],0
2,19,무얼 기다리나 (Feat. 조원선),[이영훈],[],0
3,35,찾고 있니,[이호석],[],0
4,47,Walk Alone,[LHA],[],0
...,...,...,...,...,...
79809,707922,어지러워,[헤일],[],0
79810,707925,내안의 그대,[변진섭],[],0
79811,707957,너를 그리워해,[토요],[],0
79812,707969,I Decide (Piano Remix),[헤리티지(Heritage)],[],0


In [4]:
# Load Playlist dataset
with open('./train.json', 'r', encoding='UTF8') as f:
    data = json.load(f)
pl_df = pd.json_normalize(data)

In [5]:
# Sort By like_cnt and update_date
# give weight to 'like_cnt'
pl_df.sort_values(by=['like_cnt','updt_date'], ascending=False, inplace=True)

In [6]:
pl_df.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
46524,"[노동요, 취향저격, 드라이브, Pop, 기분전환, 트렌디, 스트레스, 신나는]",74220,HOT TRENDY POP: 놓쳐선 안될 'POP' (매주 업데이트),"[159317, 368264, 600999, 274905, 643023, 24070...",53211,2020-04-17 14:32:23.000
85071,"[감성, 새벽, 때껄룩, 잔잔한, 휴식, 팝송, 떼껄룩, 힐링, 팝, 신나는]",122388,❤️때껄룩님 ᴛᴀᴋᴇ ᴀ ʟᴏᴏᴋ 플레이리스트❤️,"[532771, 383775, 432753, 176384, 152694, 60126...",41844,2020-04-23 20:58:45.000
102651,"[매장음악, 카페, 러블리쏘, 분위기, 드라이브, 여행, 매력적인, 기분전환, 팝]",26174,듣다보면 '우와!' 하고 제목을 보게되는 팝,"[40160, 20039, 561488, 20051, 16242, 256568, 7...",27268,2020-04-19 21:22:11.000
101464,"[흥겨운, 드라이브, Pop, 기분전환, 경쾌한, 외출, 내적댄스, 기분업, 충전,...",72658,"약속 있어? 외출 전, 기분 UP 하고 싶을 때 들어봐! [매주 목요일]","[111779, 406878, 417725, 129878, 357770, 34879...",23965,2020-04-22 16:05:39.000
59894,"[힐링, 휴식, 기분전환]",113501,감성이 터지는 팝음악들,"[342524, 1233, 417666, 394523, 107455, 17206, ...",23732,2015-07-29 08:25:35.000


In [38]:
def processing_playlist():
    for pl_idx, pl_row in pl_df.iterrows():
        cnt = pl_row.like_cnt
        song_list = pl_row.songs
        tags = pl_row.tags
        for song in song_list:
            if song in song_df['id'].values:
                idx, = song_df.index[song_df['id'] == song].values
                song_df.at[idx, 'count'] += 1
                origin_tags = dict(song_df.at[idx, 'tags'])
                
                for tag in tags:
                    if tag not in origin_tags: origin_tags[tag] = 1
                    elif tag in origin_tags: origin_tags[tag] += 1
                
                if len(origin_tags) > 20:
                    new_dict = {}
                    for tag, cnt in sorted(list(origin_tags.items()), key=lambda x: -x[1])[:20]:
                        new_dict[tag] = cnt
                    song_df.at[idx, 'tags'] = list(new_dict.items())
                else:
                    song_df.at[idx, 'tags'] = list(origin_tags.items())
                

In [39]:
song_df.sort_values(by='count', ascending=False, inplace=True)
song_df.head(10)

,id,song_name,artist_name_basket,tags,count
16162,144663,밤편지,[아이유],"[(감성, 690), (발라드, 543), (새벽, 339), (밤, 303), (...",4361
13048,116573,안아줘,[정준일],"[(발라드, 457), (새벽, 437), (감성, 416), (밤, 363), (...",4258
40220,357367,비,[폴킴],"[(감성, 735), (발라드, 553), (비오는날, 454), (이별, 398)...",3971
41312,366786,가끔 미치도록 네가 안고 싶어질 때가 있어,[가을방학],"[(새벽, 373), (인디, 321), (밤, 317), (감성, 294), (발...",3864
14889,133143,"그대와 나, 설레임 (Feat. 소울맨)",[어쿠스틱 콜라보],"[(기분전환, 276), (힐링, 208), (카페, 143), (발라드, 140)...",3287
76181,675115,야생화,[박효신],"[(발라드, 476), (감성, 280), (이별, 249), (추억, 220), ...",3215
39347,349492,어떤이별,[임승부],"[(감성, 659), (발라드, 628), (잔잔한, 572), (휴식, 531),...",3198
68898,610933,널 생각해,[원 모어 찬스 (one more chance)],"[(사랑, 539), (설렘, 398), (기분전환, 264), (힐링, 217),...",3096
52085,463173,비가 내렸어 (Vocal by 스티브언니),[업라이트 (Upright)],"[(감성, 662), (발라드, 635), (인디, 418), (분위기, 377),...",3088
4666,42155,벙어리,[홍아],"[(감성, 610), (발라드, 561), (카페, 494), (매장음악, 398)...",3080


,id,song_name,artist_name_basket,tags,count
16162,144663,밤편지,[아이유],"[영케이, 꿀음색, 회상, 수고했어오늘도, 분위기끝장나는, 감성적인, 입학, 창문밖...",2175
13048,116573,안아줘,[정준일],"[영케이, 내노래같은, 방탄이부른곡, 회상, 제아, 감성적인, 빨리, 매력있는, 4...",2121
40220,357367,비,[폴킴],"[영케이, 지나가는겨울, 세아, 울적함, 회상, 일상의BGM, 감성적인, bgm, ...",1981
41312,366786,가끔 미치도록 네가 안고 싶어질 때가 있어,[가을방학],"[그_냥, 울적함, 꿀음색, 회상, 수고했어오늘도, 택시, 감성적인, 청춘들의, 이...",1919
14889,133143,"그대와 나, 설레임 (Feat. 소울맨)",[어쿠스틱 콜라보],"[드라이브, 퇴근길, 혼자, 감정은그루브를타고, 회상, 꿀음색, 차트, 수제맥주, ...",1633
...,...,...,...,...,...
30222,269143,별이 되어줘,[네미시스],"[잔잔한, 밴드, 인디, 새벽, 몽환적, 한국, 얼터너티브]",1
46602,414795,Till The Night There,[안다영],"[잔잔한, 새벽, 밤]",1
29505,262640,이별을 말하다,[남영우],"[신곡, 노래, 1월]",1
30155,268471,그래서,[리마크],[발라드],1
